In [ ]:
# default_exp model
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler

# Models
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from fastai2.basics import *
from fastai2.callback.progress import *
from fastai2.text.models.qrnn import *

In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,clm_shift=[0,-1]),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,clm_shift=[-1])),
                 get_items=CreateDict([DfHDFCreateWindows(win_sz=1000+1,stp_sz=1000,clm='current')]),
                 splitter=ApplyToDict(ParentSplitter()))
db = seq.databunch(get_hdf_files('test_data/'))

## RNNs

In [ ]:
#export
class SimpleGRU(nn.Module):
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,lrn_init_state=False,**kwargs):
        super().__init__()
        self.rnn = nn.GRU(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,batch_first=True,**kwargs)
        self.final = nn.Conv1d(hidden_size,output_size,kernel_size=1)
        
        self.init_p = nn.Parameter(torch.ones(size=(num_layers,hidden_size))) if lrn_init_state else None
        
    def forward(self, x):
#         out = x.transpose(1,2)
        if self.init_p is None:
            out,_ = self.rnn(x)
        else:
            out,_ = self.rnn(x,self.init_p.expand((x.shape[0],-1,-1)).transpose(0,1).contiguous())
#         import pdb; pdb.set_trace()
        out = out.transpose(1,2)
        out = self.final(out)
        out = out.transpose(1,2)
        return out

In [ ]:
model = SimpleGRU(2,1)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,3.603175,0.440320,00:03


In [ ]:
#export
class SimpleLSTM(nn.Module):
    
    def __init__(self,input_size,output_size,num_layers=1,hidden_size=100,**kwargs):
        super().__init__()
        self.rnn = nn.LSTM(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers,batch_first=True,**kwargs)
        self.final = nn.Conv1d(hidden_size,output_size,kernel_size=1)

    def forward(self, x):
        out,_ = self.rnn(x)
        out = out.transpose(1,2)
        out = self.final(out)
        out = out.transpose(1,2)
        return out

In [ ]:
model = SimpleLSTM(2,1,num_layers=2,dropout=0.3)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,3.121614,0.236742,00:05


In [ ]:
#export
class SimpleQRNN(nn.Module):
    @delegates(QRNN, keep=True)
    def __init__(self,input_size,output_size,hidden_size=100,**kwargs):
        super().__init__()
        self.rnn = QRNN(input_size=input_size,hidden_size=hidden_size,**kwargs)
        self.final = nn.Conv1d(hidden_size,output_size,kernel_size=1)

    def forward(self, x):
        out,_ = self.rnn(x)
#         import pdb; pdb.set_trace()
        out = out.transpose(1,2)
        out = self.final(out)
        out = out.transpose(1,2)
        return out

In [ ]:
model = SimpleQRNN(2,1,window=2)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,12.073962,7.654397,00:02


## CNNs

In [ ]:
#export
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True):

        super().__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=(kernel_size - 1) * dilation,
            dilation=dilation,
            groups=groups,
            bias=bias)
        self.__padding = (kernel_size - 1) * dilation
        
    def forward(self, input):
        return super().forward(input)[:,:,:-self.__padding]

In [ ]:
#export
@delegates(CausalConv1d, keep=True)
def CConv1D(input_size,output_size,kernel_size=2,activation = MishJit, bn = True, **kwargs):
    conv = CausalConv1d(input_size,output_size,kernel_size,**kwargs)
    if bn:
        return nn.Sequential(nn.BatchNorm1d(input_size,eps=1e-3, momentum=0.01),conv,activation())
    else:
        return nn.Sequential(conv,activation())

In [ ]:
#export
class TCN(nn.Module):
    def __init__(self,input_size,output_size,hl_depth=1,hl_width=10,act = MishJit):
        super().__init__()
        
        
        self.hl_depth = hl_depth

        self.conv1 = CConv1D(input_size,hl_width)
        self.conv_layers = nn.ModuleList([CConv1D(hl_width,hl_width,dilation=2**(i+1)) for i in range(hl_depth-1)])
        
        self.final = nn.Conv1d(hl_width,output_size,kernel_size=1)
        
    def forward(self, x):
        out = x.transpose(1,2)
        out = self.conv1(out)
        for i in range(self.hl_depth-1):
            out = self.conv_layers[i](out)+out
#         print(out.shape)
#         print(self.final.in_features)
        out = self.final(out)
        out = out.transpose(1,2)
        return out

In [ ]:
model = TCN(2,1)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,3.169277,2.073140,00:01


## CRNNs

In [ ]:
#export
def ConvGRU(input_size,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=1,hs_cnn=10,hs_rnn=100):
    cnn = TCN(input_size,num_ft,hl_depth=num_cnn_layers,hl_width=hs_cnn)
    rnn = SimpleGRU(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn)
    return nn.Sequential(cnn,rnn)

In [ ]:
model = ConvGRU(2,1,10)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,3.260091,0.239342,00:03


In [ ]:
#export
def ConvQRNN(input_size,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=1,hs_cnn=10,hs_rnn=100):
    cnn = TCN(input_size,num_ft,hl_depth=num_cnn_layers,hl_width=hs_cnn)
    rnn = SimpleQRNN(num_ft,output_size,n_layers=num_rnn_layers,hidden_size=hs_rnn)
    return nn.Sequential(cnn,rnn)

In [ ]:
model = ConvQRNN(2,1,10)
lrn = Learner(db,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,4.018788,0.271553,00:02


## ProDiag

In [ ]:
#export
class ProgDiag_GRU(nn.Module):
    def __init__(self,input_size,output_size,init_size,hidden_size=100,hidden_layer = 1):
        super().__init__()
        self.out_sz = output_size
        self.init_size = init_size
        self.diag_state = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                               num_layers=1, batch_first=True)
        self.prog_state = nn.GRU(input_size=input_size-1,hidden_size=hidden_size,
                               num_layers=1, batch_first=True)

        self.estimator = SimpleGRU(input_size=hidden_size,output_size=output_size,hidden_size=hidden_size,
                           num_layers=hidden_layer)
             
        self.final = nn.Conv1d(hidden_size,self.out_sz,kernel_size=1)

    def forward(self, x,init_state = None):
        h_diag,_ = self.diag_state(x)   
        
        h_prog_init = h_diag[:,self.init_size-self.out_sz,:]
        h_prog,_ = self.prog_state(x[:,self.init_size:,:-self.out_sz],h_prog_init[None,...])
        
        est_diag = self.estimator(h_diag)[:,self.init_size:]
        est_prog = self.estimator(h_prog)
        
        
        out = torch.cat([est_diag,est_prog],dim=2)
#         import pdb; pdb.set_trace()   
        return out, h_diag[:,self.init_size:,:],h_prog

In [ ]:
model = ProgDiag_GRU(2,1,100)
lrn = Learner(db,model,loss_func=nn.MSELoss(),cbs=ProDiagTrainer(100)).fit(1)

epoch,train_loss,valid_loss,time
0,3.266276,0.290311,00:07


In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_model.ipynb.
Converted 02_learner.ipynb.
Converted 10_performance_test.ipynb.
Converted 90_imu_example.ipynb.
Converted 91_cascaded_tanks.ipynb.
Converted 92_wiener_hammerstein.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 93_mdt_benchmark.ipynb.
Converted 93_noisyhammerstein.ipynb.
Converted 93_silverbox.ipynb.
Converted index.ipynb.
